Este notebook tem o objetivo de apresentar uma forma de realizar classificação textual utilizando redes neurais profundas.

Será realizado um exemplo bem simples para depois aplicar a metodologia em uma base de maior volume, mais próxima da realidade.

O exemplo inicial foi baseado no post:  https://medium.com/analytics-vidhya/understanding-embedding-layer-in-keras-bbe3ff1327ce .

In [ ]:
# Importação das bibliotecas

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import one_hot
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

Neste primeiro exemplo, serão consideradas as seguintes expressões que poderiam ser encontradas como comentários a respeito da qualidade de um trabalho.

In [ ]:
docs = ['Ótimo trabalho!',
		'Muito bom',
		'Excelente resultado',
		'belo trabalho',
		'Excelente!',
		'Fraco!',
		'Podia ser melhor.',
		'faltou empenho',
		'Não ficou bom',
		'Abaixo do esperado.']
# Definição dos labels (1: positivo; 0: negativo)
labels = array([1,1,1,1,1,0,0,0,0,0])

A primeira etapa é codificar o vocabulário em um id único.
Define-se um range de números e com a função `one_hot` "espalha" as palavras no intervalo de modo que não ocorram colisões.  

In [ ]:
# Define-se um tamanho arbitrário
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
# Documentos codificados
print(encoded_docs)

[[30, 38], [20, 29], [17, 30], [36, 38], [17], [40], [17, 4, 21], [20, 43], [9, 27, 29], [19, 16, 14]]


A segunda etapa é a de `padding` que nada mais é que a padronização do tamanho dos documentos de entrada. Quando o tamanho máximo é atingido, a parte susequente é desconsiderada. Caso o documento tenha tamanho menor, o restante é complementado com 0's.

In [ ]:
max_length = 3
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[30 38  0]
 [20 29  0]
 [17 30  0]
 [36 38  0]
 [17  0  0]
 [40  0  0]
 [17  4 21]
 [20 43  0]
 [ 9 27 29]
 [19 16 14]]


Após a etapa de pré-processamento, chega o momento de definir a arquitetura da rede.

In [ ]:
# Arquitetura
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compilação do modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Sumarização da arquitetura
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 8)              400       
                                                                 
 flatten (Flatten)           (None, 24)                0         
                                                                 
 dense (Dense)               (None, 1)                 25        
                                                                 
Total params: 425
Trainable params: 425
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Treinamento da rede
model.fit(padded_docs, labels, epochs=50, verbose=1)
# Avaliação do modelo
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 0.6967 - accuracy: 0.4000
Epoch 2/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6945 - accuracy: 0.4000
Epoch 3/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6922 - accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6900 - accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6878 - accuracy: 0.6000
Epoch 6/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6855 - accuracy: 0.6000
Epoch 7/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6833 - accuracy: 0.6000
Epoch 8/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6811 - accuracy: 0.8000
Epoch 9/50
1/1 [==============================] - 0s 12ms/step - loss: 0.6789 - accuracy: 0.8000
Epoch 10/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6767 - accuracy: 0.8000
Epoch 11/50
1/1 [====================

## Utilizando um modelo de embeddings pré-treinado

In [ ]:
# Utilizando um modelo pré-treinado
!wget http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s50.zip
!unzip /content/download.php?file=embeddings%2Fword2vec%2Fcbow_s50.zip

--2022-11-04 23:57:52--  http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s50.zip
Connecting to 143.107.183.175:22980... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170360268 (162M) [application/octet-stream]
Saving to: ‘download.php?file=embeddings%2Fword2vec%2Fcbow_s50.zip’

download.php?file=e 100%[===================>] 162.47M  6.38MB/s    in 40s     

2022-11-04 23:58:32 (4.07 MB/s) - ‘download.php?file=embeddings%2Fword2vec%2Fcbow_s50.zip’ saved [170360268/170360268]

Archive:  /content/download.php?file=embeddings%2Fword2vec%2Fcbow_s50.zip
  inflating: cbow_s50.txt            


In [ ]:
# Para a codificação dos tokens, pode ser utilizada a classe 'Tokenizer'
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

In [ ]:
t.word_index

{'trabalho': 1,
 'bom': 2,
 'excelente': 3,
 'ótimo': 4,
 'muito': 5,
 'resultado': 6,
 'belo': 7,
 'fraco': 8,
 'podia': 9,
 'ser': 10,
 'melhor': 11,
 'faltou': 12,
 'empenho': 13,
 'não': 14,
 'ficou': 15,
 'abaixo': 16,
 'do': 17,
 'esperado': 18}

In [ ]:
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
import numpy as np

# Preprocessamento dos dados
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

encoded_docs = t.texts_to_sequences(docs)

# Etapa de padding dos documentos
max_length = 3
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

Quando se utiliza uma camada de Embedding já treinada, é preciso criar a matriz de Embeddings.

In [ ]:
# Carrega a matriz com as informações do modelo
embeddings_index = dict()
f = open('/content/cbow_s50.txt')
for line in f:
  values = line.split()
  word = values[0]
  try:
    coefs = asarray(values[1:], dtype='float32')
  except:
    coefs = np.zeros(50)
  embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# Cria a matriz de Embeddings utilizando as informações do modelo
embedding_matrix = zeros((vocab_size, 50))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 929595 word vectors.


In [ ]:
embedding_matrix

In [ ]:
# Criação da camada de Embeddings
e = Embedding(vocab_size, 50, weights=[embedding_matrix], input_length=3, trainable=False)

In [ ]:
# Definição da arquitetura
model = Sequential()
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# Compilação do modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Sumarização da arquitetura
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 50)             950       
                                                                 
 flatten_1 (Flatten)         (None, 150)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 151       
                                                                 
Total params: 1,101
Trainable params: 151
Non-trainable params: 950
_________________________________________________________________
None


In [ ]:
# Treinamento da rede
model.fit(padded_docs, labels, epochs=50, verbose=0)

In [ ]:
# Avaliação do modelo
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 89.999998


## Aplicando sobre um dataset maior para a classificação de críticas de filmes

Após aplicar a metodologia sobre um `toy example`, vamos aplicar em um dataset de críticas de filmes fazendo a análise de sentimento.

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/TopicosNLP-02_2022/Notebooks/Transformes/imdb-reviews-pt-br.csv.zip')

In [ ]:
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [ ]:
df.shape

(49459, 4)

In [ ]:
# Para simplificar mais o experimento, vamso pegar um conjunto menor do dataset.
df_pos=df[df.sentiment=='pos'].sample(5000, random_state=42)
df_neg=df[df.sentiment=='neg'].sample(5000, random_state=42)
df=pd.concat([df_pos,df_neg], ignore_index=True)
df.head()

,id,text_en,text_pt,sentiment
0,37528,ROLL is a wonderful little film. Toby Malone p...,ROLL é um pequeno filme maravilhoso. Toby Malo...,pos
1,44113,This is a great movie that I dont think gets e...,Este é um ótimo filme que eu acho que não rece...,pos
2,21351,The original story and funny compelling charac...,A história original e os personagens convincen...,pos
3,16267,I am very impressed by the reviews Ive read of...,Estou muito impressionado com os comentários q...,pos
4,45604,"Well, of course not, women are overly sensitiv...","Bem, claro que não, as mulheres são excessivam...",pos


In [ ]:
# Necessário replicar a descrição do sentimento para números inteiros
sentiment_to_code = {'neg':0, 'pos':1}
df.replace({'sentiment': sentiment_to_code}, inplace=True)

#### Utilizando a camada no Keras

In [ ]:
# Separando os dados em treinamento e teste
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text_pt,
                                                    df.sentiment,
                                                    test_size=0.2,
                                                    stratify=df.sentiment,
                                                    random_state=42)

docs=X_train
labels=y_train

In [ ]:
# O primeiro passo do pré-processamento é sempre a decodificação dos tokens
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index)

In [ ]:
import pickle

with open('file.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(t, file)

In [ ]:
t.word_index

In [ ]:
df.text_pt.apply(len).max()

5335

In [ ]:
# A segunda etapa é o de padding, etapa que possibilita a redução da 
# dimensionalidade do modelo conforme necessidade

encoded_docs = t.texts_to_sequences(docs)
max_length = 500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
# A etapa seguinte é a criação da rede
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 50)           2980850   
                                                                 
 flatten_3 (Flatten)         (None, 25000)             0         
                                                                 
 dense_3 (Dense)             (None, 1)                 25001     
                                                                 
Total params: 3,005,851
Trainable params: 3,005,851
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Treinamento da rede
model.fit(padded_docs, labels, epochs=10, verbose=1)

Epoch 1/10
250/250 [==============================] - 1s 4ms/step - loss: 0.6479 - accuracy: 0.6235
Epoch 2/10
250/250 [==============================] - 1s 4ms/step - loss: 0.2736 - accuracy: 0.9229
Epoch 3/10
250/250 [==============================] - 1s 4ms/step - loss: 0.0766 - accuracy: 0.9883
Epoch 4/10
250/250 [==============================] - 1s 4ms/step - loss: 0.0274 - accuracy: 0.9986
Epoch 5/10
250/250 [==============================] - 1s 4ms/step - loss: 0.0130 - accuracy: 0.9999
Epoch 6/10
250/250 [==============================] - 1s 4ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 7/10
250/250 [==============================] - 1s 4ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 8/10
250/250 [==============================] - 1s 4ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 9/10
250/250 [==============================] - 1s 6ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 10/10
250/250 [==============================] - 2s 7ms/step - loss: 0.0017 - accuracy: 1.0000

In [ ]:
# Avaliação da rede utilizando os dados de teste
encoded_docs_test = t.texts_to_sequences(X_test)
max_length = 500
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

loss, accuracy = model.evaluate(padded_docs_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 85.450000


### Para avançar mais...

1. Como poderíamos testar novos valores neste modelo???
2. Não foram utilizados recursos de normalização... Será que eles fariam diferença no resultado final da acurácia do modelo???

In [ ]:
df_bruto = pd.read_csv('/content/drive/MyDrive/TopicosNLP-02_2022/Notebooks/Transformes/imdb-reviews-pt-br.csv.zip')

In [ ]:
df_bruto.sample(1)

,id,text_en,text_pt,sentiment
24008,24010,"Although I am not a Michael Jackson fan, I lik...","Embora eu não seja um fã de Michael Jackson, e...",pos


In [ ]:
novo_teste = df_bruto.loc[15851].text_pt

In [ ]:
novo_teste

'Dolemite pode não ter sido o primeiro filme de exploração negra a aparecer, mas certamente é um dos melhores. É um filme crucial no gênero Black Exploitation, onde causou uma mudança dramática entre os filmes que vieram antes dele, em contraste com os filmes que vieram depois dele. Não foi necessariamente um filme comovente ou comovente sobre a cultura negra e sua luta para superar questões como o racismo ou qualquer coisa tão importante quanto isso, mas foi a história de um cara malvado lutando "whitey" com seu exército de mamas quentes kung-fu . Foi um prazer culpado, muito divertido e melhor assisti-lo com os amigos. 10 de 10'

In [ ]:
# para jogar no modelo

with open('/content/file.pkl', 'rb') as file:
    model_tokenizer = pickle.load(file)

novo_teste_encoded = model_tokenizer.texts_to_sequences([novo_teste])

In [ ]:
novo_teste_padded = pad_sequences(novo_teste_encoded, maxlen=max_length, padding='post')

In [ ]:
model.predict(novo_teste_padded)

1/1 [==============================] - 0s 41ms/step


array([[0.87655777]], dtype=float32)